In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src')

In [3]:
from parse_cesar1 import get_timestep_tuples
import pickle

In [4]:
from pyprocessta.causalimpact import _select_unrelated_x
from pyprocessta.model.tcn import transform_data, run_model

In [5]:
from darts import TimeSeries
import pandas as pd
from definitions import TARGETS
import matplotlib.pyplot as plt

MEAS_COLUMNS = [
    "TI-19",
    "FI-16",
    "TI-33",
    "FI-2",
    "FI-151",
    "TI-8",
    "FI-241",
    "valve-position-12",  # dry-bed
    "FI-38",  # stripper
    "PI-28",  # stripper
    "TI-28",  # stripper
    "FI-20",
    "FI-30",
    "TI-3",
    "FI-19",
    "FI-211",
    "FI-11",
    "TI-30",
    "PI-30",
    "TI-1213",
    "TI-4",
    "FI-23",
    "delta_t",
]

In [6]:
df = pd.read_pickle('detrended.pkl')
df = df.dropna()
df['delta_t'] = df['TI-35'] - df["TI-4"]
MEAS_COLUMNS.append('delta_t')

In [7]:
plt.plot(df['delta_t'])

In [8]:
with open('step_times.pkl', 'rb') as handle:
    times = pickle.load(handle)

In [9]:
a, b = get_timestep_tuples(df, times, 0)

In [10]:
a

[Timestamp('2020-01-15 11:44:01', freq='2T'),
 Timestamp('2020-01-15 14:08:01', freq='2T')]

In [11]:
def get_causalimpact_splits(x, y, day, times, df): 
    a, b = get_timestep_tuples(df, times, day)
 
    x_way_before, x_after = x.split_before(a[1])
    y_way_before, y_after = y.split_before(pd.Timestamp(a[1]))

    _, x_before = x_way_before.split_before(a[0])
    _, y_before = y_way_before.split_before(a[0])

    x_during, x_test = x_after.split_before(b[1])
    y_during, y_test = y_after.split_before(b[1])

    return (x_before, y_before), (x_during, y_during), (x_test, y_test)

    
    

In [12]:
y = TimeSeries.from_dataframe(df, value_cols=TARGETS)
x = TimeSeries.from_dataframe(df, value_cols=MEAS_COLUMNS)

In [13]:
before, during, after = get_causalimpact_splits(x, y, 1, times, df)

In [14]:
train_tuple, test_tuple, scalers = transform_data(before, [during, after])

In [15]:
run_model(train_tuple)

In [16]:
run_model(train_tuple)

In [17]:
run_model(train_tuple)

In [18]:
model = run_model(train_tuple)

In [19]:
from pyprocessta.causalimpact import _select_unrelated_x
from pyprocessta.model.tcn import transform_data, run_model, enable_dropout

In [20]:
enable_dropout(model)

In [21]:
x_test, y_test = test_tuple    
backtest_cov = model.historical_forecasts(y_test,
                                                covariates=x_test,
                                                start=0.05,
                                                forecast_horizon=1,
                                                stride=1,
                                                retrain=False,
                                                verbose=False)

In [22]:
y_test

(                            0          1         2         3          4  \
 2020-01-16 13:06:01  0.450483  -0.122026  0.656615  0.593754   1.163361   
 2020-01-16 13:08:01  0.453592  -0.136000  0.676168  0.582453   1.116737   
 2020-01-16 13:10:01  0.463435  -0.082740  0.663647  0.675735   1.218284   
 2020-01-16 13:12:01  0.468349  -0.076349  0.666320  0.775382   1.172681   
 2020-01-16 13:14:01  0.464168  -0.181757  0.653047  0.850054   1.155539   
 ...                       ...        ...       ...       ...        ...   
 2020-02-01 23:50:01  0.396024 -12.056767 -3.131103  1.265446  14.608552   
 2020-02-01 23:52:01  0.422290 -12.052093 -3.169131  1.427462  14.634171   
 2020-02-01 23:54:01  0.457059 -12.037981 -3.209145  1.275299  14.642683   
 2020-02-01 23:56:01  0.471380 -12.023690 -3.251139  1.205790  14.568009   
 2020-02-01 23:58:01  0.493769 -12.031640 -3.246436  1.148783  14.538586   
 
                             5         6          7         8         9  ...  \
 2020-

In [23]:
train_tuple, test_tuple, scalers = transform_data(before, [during, after])

In [24]:
y_test

(                            0          1         2         3          4  \
 2020-01-16 13:06:01  0.450483  -0.122026  0.656615  0.593754   1.163361   
 2020-01-16 13:08:01  0.453592  -0.136000  0.676168  0.582453   1.116737   
 2020-01-16 13:10:01  0.463435  -0.082740  0.663647  0.675735   1.218284   
 2020-01-16 13:12:01  0.468349  -0.076349  0.666320  0.775382   1.172681   
 2020-01-16 13:14:01  0.464168  -0.181757  0.653047  0.850054   1.155539   
 ...                       ...        ...       ...       ...        ...   
 2020-02-01 23:50:01  0.396024 -12.056767 -3.131103  1.265446  14.608552   
 2020-02-01 23:52:01  0.422290 -12.052093 -3.169131  1.427462  14.634171   
 2020-02-01 23:54:01  0.457059 -12.037981 -3.209145  1.275299  14.642683   
 2020-02-01 23:56:01  0.471380 -12.023690 -3.251139  1.205790  14.568009   
 2020-02-01 23:58:01  0.493769 -12.031640 -3.246436  1.148783  14.538586   
 
                             5         6          7         8         9  ...  \
 2020-

In [25]:
x_test, y_test = test_tuple    
backtest_cov = model.historical_forecasts(y_test,
                                                covariates=x_test,
                                                start=0.05,
                                                forecast_horizon=1,
                                                stride=1,
                                                retrain=False,
                                                verbose=False)

In [26]:
type(y_test)

tuple

In [27]:
x_test, y_test = test_tuple[0]    
backtest_cov = model.historical_forecasts(y_test,
                                                covariates=x_test,
                                                start=0.05,
                                                forecast_horizon=1,
                                                stride=1,
                                                retrain=False,
                                                verbose=False)

In [28]:
x_test

                            0         1         2         3         4  \
2020-01-16 09:08:01 -0.361016  0.105076  0.455924  0.417780  0.901699   
2020-01-16 09:10:01 -0.295432  0.100611  0.492838  0.555943  0.773510   
2020-01-16 09:12:01 -0.415280  0.160958  0.493131  0.750142  0.823246   
2020-01-16 09:14:01 -0.400701  0.176177  0.488914  0.493393  0.903302   
2020-01-16 09:16:01 -0.367976  0.194937  0.460158  0.324511  0.844472   
...                       ...       ...       ...       ...       ...   
2020-01-16 12:56:01  0.426576 -0.024591  0.707169  0.520700  1.242417   
2020-01-16 12:58:01  0.427609 -0.065799  0.710318  0.681427  1.129127   
2020-01-16 13:00:01  0.447991 -0.180700  0.687218  0.684807  1.239125   
2020-01-16 13:02:01  0.449971 -0.055551  0.654962  0.777797  1.087663   
2020-01-16 13:04:01  0.447923 -0.097887  0.656591  0.688682  1.236709   

                            5         6         7         8         9  ...  \
2020-01-16 09:08:01  0.938085  0.000859  1.00

In [29]:
y_test

                            0         1         2         3
2020-01-16 09:08:01  0.353501  0.083769  0.329059 -0.002288
2020-01-16 09:10:01  0.393777  0.076587  0.317271 -0.004577
2020-01-16 09:12:01  0.635726  0.061878  0.274861 -0.006865
2020-01-16 09:14:01  0.748604  0.043835  0.237942 -0.009153
2020-01-16 09:16:01  0.748545  0.064994  0.312420 -0.011442
...                       ...       ...       ...       ...
2020-01-16 12:56:01  0.266064  0.122714  0.088354  2.150492
2020-01-16 12:58:01  0.225670  0.096032  0.051963  2.134941
2020-01-16 13:00:01  0.306280  0.104182  0.065516  2.013297
2020-01-16 13:02:01  0.338488  0.097253  0.052354  1.997747
2020-01-16 13:04:01  0.386831  0.089086  0.034019  1.862840

[119 rows x 4 columns]
Freq: 2T

In [30]:
x_test, y_test = train_tuple  
backtest_cov = model.historical_forecasts(y_test,
                                                covariates=x_test,
                                                start=0.05,
                                                forecast_horizon=1,
                                                stride=1,
                                                retrain=False,
                                                verbose=False)

In [31]:
backtest_cov.plot()

In [32]:
y_train.plot(label='actual')
backtest_cov.plot(label='forecast')

In [33]:
y_test.plot(label='actual')
backtest_cov.plot(label='forecast')

In [34]:
y_test.plot(label='actual')
backtest_cov.plot(label='forecast')
plt.legend()

In [35]:
train_tuple, test_tuple, scalers = transform_data(before, [during, after])

In [36]:
test_tuple[0]

(                            0         1         2         3         4  \
 2020-01-16 09:08:01 -0.361016  0.105076  0.455924  0.417780  0.901699   
 2020-01-16 09:10:01 -0.295432  0.100611  0.492838  0.555943  0.773510   
 2020-01-16 09:12:01 -0.415280  0.160958  0.493131  0.750142  0.823246   
 2020-01-16 09:14:01 -0.400701  0.176177  0.488914  0.493393  0.903302   
 2020-01-16 09:16:01 -0.367976  0.194937  0.460158  0.324511  0.844472   
 ...                       ...       ...       ...       ...       ...   
 2020-01-16 12:56:01  0.426576 -0.024591  0.707169  0.520700  1.242417   
 2020-01-16 12:58:01  0.427609 -0.065799  0.710318  0.681427  1.129127   
 2020-01-16 13:00:01  0.447991 -0.180700  0.687218  0.684807  1.239125   
 2020-01-16 13:02:01  0.449971 -0.055551  0.654962  0.777797  1.087663   
 2020-01-16 13:04:01  0.447923 -0.097887  0.656591  0.688682  1.236709   
 
                             5         6         7         8         9  ...  \
 2020-01-16 09:08:01  0.938085 

In [37]:
train_tuple[1]

                            0         1         2         3
2020-01-15 18:32:01  0.218164  0.405769  0.011118  1.000000
2020-01-15 18:34:01  0.266507  0.414905  0.088341  0.997712
2020-01-15 18:36:01  0.274514  0.347606  0.000000  0.995423
2020-01-15 18:38:01  0.314790  0.400718  0.040161  0.993135
2020-01-15 18:40:01  0.266329  0.598488  0.140509  0.990847
...                       ...       ...       ...       ...
2020-01-16 08:58:01  0.345731  0.055065  0.248941  0.009153
2020-01-16 09:00:01  0.394074  0.035961  0.199562  0.006865
2020-01-16 09:02:01  0.361746  0.059141  0.241730  0.004577
2020-01-16 09:04:01  0.353620  0.073505  0.288544  0.002288
2020-01-16 09:06:01  0.385828  0.079256  0.329339  0.000000

[438 rows x 4 columns]
Freq: 2T

In [38]:
x_test, y_test = test_tuple[0]  
backtest_cov = model.historical_forecasts(y_test,
                                                covariates=x_test,
                                                start=0.05,
                                                forecast_horizon=1,
                                                stride=1,
                                                retrain=False,
                                                verbose=False)

In [39]:
len(y_test)

119

In [40]:
len(x_test)

119

In [41]:
x_test, y_test = test_tuple[1]  
backtest_cov = model.historical_forecasts(y_test,
                                                covariates=x_test,
                                                start=0.05,
                                                forecast_horizon=1,
                                                stride=1,
                                                retrain=False,
                                                verbose=False)

In [42]:
y_test.plot(label='actual')
backtest_cov.plot(label='forecast')
plt.legend()

In [43]:
x_test, y_test = test_tuple[1]  
backtest_cov = model.historical_forecasts(y_test,
                                                covariates=x_test,
                                                start=0.01,
                                                forecast_horizon=1,
                                                stride=1,
                                                retrain=False,
                                                verbose=False)

In [44]:
x_test, y_test = test_tuple[0]  
backtest_cov = model.historical_forecasts(y_test,
                                                covariates=x_test,
                                                start=0.01,
                                                forecast_horizon=1,
                                                stride=1,
                                                retrain=False,
                                                verbose=False)

In [45]:
x_test, y_test = test_tuple[0]  
backtest_cov = model.historical_forecasts(y_test,
                                                covariates=x_test,
                                                start=0.1,
                                                forecast_horizon=1,
                                                stride=1,
                                                retrain=False,
                                                verbose=False)

In [46]:
y_test.plot(label='actual')
backtest_cov.plot(label='forecast')
plt.legend()

In [47]:
x_test, y_test = test_tuple[0]  
backtest_cov = model.historical_forecasts(y_test,
                                                covariates=x_test,
                                                start=0.05,
                                                forecast_horizon=1,
                                                stride=1,
                                                retrain=False,
                                                verbose=False)

In [48]:
x_test, y_test = test_tuple[0]  
backtest_cov = model.historical_forecasts(y_test,
                                                covariates=x_test,
                                                start=0.1,
                                                forecast_horizon=1,
                                                stride=1,
                                                retrain=False,
                                                verbose=False)

In [49]:
y_test.plot(label='actual')
backtest_cov.plot(label='forecast')
plt.legend()

In [50]:
x_test, y_test = test_tuple[0]  
backtest_cov = model.historical_forecasts(y_test,
                                                covariates=x_test,
                                                start=0.1,
                                                forecast_horizon=1,
                                                stride=1,
                                                retrain=False,
                                                verbose=False)

In [51]:
y_test.plot(label='actual')
backtest_cov.plot(label='forecast')
plt.legend()

In [52]:
x_test, y_test = test_tuple[0]  
backtest_cov = model.historical_forecasts(y_test,
                                                covariates=x_test,
                                                start=0.1,
                                                forecast_horizon=1,
                                                stride=1,
                                                retrain=False,
                                                verbose=False)

In [53]:
y_test.plot(label='actual')
backtest_cov.plot(label='forecast')
plt.legend()

In [54]:
x_test, y_test = test_tuple[0]  
backtest_cov = model.historical_forecasts(y_test,
                                                covariates=x_test,
                                                start=0.1,
                                                forecast_horizon=1,
                                                stride=1,
                                                retrain=False,
                                                verbose=False)

In [55]:
y_test.plot(label='actual')
backtest_cov.plot(label='forecast')
plt.legend()

In [56]:
enable_dropout(model)

In [57]:
x_test, y_test = test_tuple[0]  
backtest_cov = model.historical_forecasts(y_test,
                                                covariates=x_test,
                                                start=0.1,
                                                forecast_horizon=1,
                                                stride=1,
                                                retrain=False,
                                                verbose=False)

In [58]:
y_test.plot(label='actual')
backtest_cov.plot(label='forecast')
plt.legend()

In [59]:
x_test, y_test = test_tuple[0]  
backtest_cov = model.historical_forecasts(y_test,
                                                covariates=x_test,
                                                start=0.1,
                                                forecast_horizon=1,
                                                stride=1,
                                                retrain=False,
                                                verbose=False)

In [60]:
y_test.plot(label='actual')
backtest_cov.plot(label='forecast')
plt.legend()

In [61]:
x_test, y_test = test_tuple[0]  
backtest_cov = model.historical_forecasts(y_test,
                                                covariates=x_test,
                                                start=0.1,
                                                forecast_horizon=1,
                                                stride=1,
                                                retrain=False,
                                                verbose=False)

In [62]:
y_test.plot(label='actual')
backtest_cov.plot(label='forecast')
plt.legend()

In [63]:
before, during, after = get_causalimpact_splits(x, y, 0, times, df)

In [64]:
train_tuple, test_tuple, scalers = transform_data(before, [during, after])

In [65]:
model = run_model(train_tuple)